In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gc

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
%%time
# Load BES Panel for Wave 8
BES_Panel = pd.read_stata("../BES_analysis_data/BES2015_W8_v1.3.dta")
print( BES_Panel.shape )

(31409, 733)
Wall time: 13 s


In [3]:
# Select only respondents from England who answer Stay/Remain to euVote
England = (BES_Panel["country"]=="England").values
print ( "respondents not from England: ", sum(BES_Panel["country"]!="England") )
Remain  = (BES_Panel["euRefVote"].cat.codes==0).values
Leave   = (BES_Panel["euRefVote"].cat.codes==1).values
print ( "respondents not from England: ", len(BES_Panel[~(Remain | Leave)]) )
BES_reduced = BES_Panel[England & (Remain | Leave)]
print( BES_reduced.shape )

respondents not from England:  11944
respondents not from England:  2344
(17979, 733)


In [4]:
variable_categories = pd.read_csv("question_categories_correct.csv", encoding = "ISO-8859-1")

# flipping list
var_cat_dict = dict()
for typ in [1,2,3,4,5,6]:

    e = variable_categories[variable_categories.type==typ]["column_name"].values

    var_cat_dict[typ] = [item for sublist in [i.split("|") for i in e] for item in sublist]
    
var_cat_dict

{1: ['responsibleDebtConservativesW1',
  'responsibleDebtLiberalDemsW1',
  'responsibleDebtLabourW1',
  'responsibleDebtUKBanksW1',
  'responsibleDebtGlobalBanksW1',
  'responsibleDebtTradeUnionsW1',
  'responsibleDebtEUW1',
  'responsibleDebtOtherW1',
  'responsibleDebtNoneW1',
  'responsibleEconConservativesW1',
  'responsibleEconLibDemsW1',
  'responsibleEconLabourW1',
  'responsibleEconScottishGovtW1',
  'responsibleEconWelshGovtW1',
  'responsibleEconNoneW1',
  'responsibleNHSConservativesW1',
  'responsibleNHSLibDemsW1',
  'responsibleNHSLabourW1',
  'responsibleNHSScottishGovtW1',
  'responsibleNHSWelshGovtW1',
  'responsibleNHSNoneW1',
  'responsibleEducationConW1',
  'responsibleEducationLDW1',
  'responsibleEducationLabW1',
  'responsibleEducationScotGovtW1',
  'responsibleEducationWelshGovtW1',
  'responsibleEducationNoneW1',
  'responsibleCostLiveConW1',
  'responsibleCostLiveLDW1',
  'responsibleCostLiveLabW1',
  'responsibleCostLiveScotGovtW1',
  'responsibleCostLiveWelsh

In [5]:
var_type = pd.DataFrame(columns = ['type'] )
# df = DataFrame(columns=('lib', 'qty1', 'qty2'))
for col in BES_reduced.columns:
    dt = BES_reduced[col].dtype.name
    if ( dt!='category' ) and ( dt!='float64' ):
        var_type.loc[col] = -1
    
    elif ( dt=='float64' ) and (col in ['personality_agreeableness', 'personality_conscientiousness','personality_extraversion', 'personality_neuroticism','personality_openness','mapNamesW3','riskScaleW8']):
        var_type.loc[col] = 0

    elif ( dt=='float64' ):
        var_type.loc[col] = -1

    elif ( dt=='category' ):
        for typ in [1,2,3,4,5,6]:
            if col in var_cat_dict[typ]:
                var_type.loc[col] = typ

    else:
        print("what's up with this? " + col)
var_type["type"] = var_type["type"].astype("int8")
var_type

,type
id,-1
wt_core_W8,-1
wt_core_W9,-1
wt_full_W8,-1
wt_full_W1W2W3W4W5W6W7,-1
wt_full_W7,-1
wt_full_W9,-1
wt_full_W1W2W3W4W5W6W7W8W9,-1
wt_full_W7W8W9,-1
wave9,-1


In [282]:
"Larger|Smaller|About the same|Don't know".split("|")

['Larger', 'Smaller', 'About the same', "Don't know"]

In [406]:
%%time
# changing the order of some sets of categories
change_cat_dict = {"Bad time to buy|Good time to buy|Neither good nor bad time to buy|Don't know": ["Bad time to buy", "Neither good nor bad time to buy", "Good time to buy"],
                   "Larger|Smaller|About the same|Don't know": ["Larger", "About the same", "Smaller"],
                   "Mainly leave|Mainly remain|Fairly evenly split|Don't know": ["Mainly remain", "Fairly evenly split", "Mainly leave", "Don't know"],
                   'An individual share in a company|A portfolio of different company shares|The risk is the same|Don\x92t know|Prefer not to say': ['An individual share in a company', 'The risk is the same', 'A portfolio of different company shares'],
                   "No, I have never been a member|Yes, I am a member of a party|I am not a member now but I used to be|Don't know": ['No, I have never been a member', 'I am not a member now but I used to be', 'Yes, I am a member of a party'],
                   "Never or practically never|Less often than once a year|Less often but at least once a year|Less often but at least twice a year|Less often but at least once a month|Less often but at least once in two weeks|Once a week or more|Varies too much to say|I am not religious|Don't know": ['I am not religious', 'Never or practically never', 'Less often than once a year', 'Less often but at least once a year', 'Less often but at least twice a year', 'Less often but at least once a month', 'Less often but at least once in two weeks', 'Once a week or more']
                  }

                   
Weasel_answers = ["Don't know", 'Do\x92t know', 'Dont know', 'Donât know', 'Don??t know',
                  "Prefer not to say", "Prefer not to answer", "Refused", "Unknown",
                  "Neither", "Other", "I would/will not vote", "Will not vote",
                  "I would not vote", "It depends", "Other",
                  "Don’t follow politics on Facebook", "Don't follow politics on twitter",
                  "Yes, other", "Haven't thought about it",
                  "There wasn't a local election in my area", "No, haven't received it",
                  "I don't know what was negotiated", "I never received a response",
                  "There are not local elections in my area", "Can't remember",
                  "Varies too much to say", "Will not state a choice",
                  "All leaders equally good", "They are not eligible to vote",
                  "There are not local elections in my area"]

Weasel_number_answers = [ "9999.0", "997.0", "222.0", "99.0", "0.0" ]

# ADD
# 9999.0
# 99.0 - unfortunately, it also appears in some numerical answers
# Probably need to have a separate check - e.g. is the preceding category "98.0"/"98"

# non-answer answers
Weasel_set = set(Weasel_answers) # gets rid of duplicates!


Wall time: 0 ns


In [12]:
%%time
BES_numeric = BES_reduced.copy()
pos = 0
for col in BES_numeric:
    print(100*pos/len(BES_numeric.columns))
    pos = pos + 1
    if col not in var_type["type"].index:
        print(col, " not in var_type")
        continue
    typ = var_type["type"][col]
    if (typ == -1) | (typ == 4) | (typ == 3) | (col =="id"): # skip
        continue
        #BES_numeric.drop(col,axis=1,inplace=True)
        
    
    elif (typ == 2):
        new_cat_order = change_cat_dict["|".join(BES_numeric[col].cat.categories)]
        BES_numeric[col].cat.remove_categories( BES_numeric[col].cat.categories.intersection(Weasel_set), inplace=True )
        BES_numeric[col].cat.reorder_categories( new_cat_order, inplace=True )

        BES_numeric[col] = BES_numeric[col].cat.codes
#        BES_numeric.loc[ BES_numeric[col]==-1, col ] = np.nan  
    
    elif (typ == 5) | (typ == 1): # (typ == 1) | 
        # remove weasel categories
        BES_numeric[col].cat.remove_categories( BES_numeric[col].cat.categories.intersection(Weasel_set), inplace=True )
        # change to numbers!
        BES_numeric[col] = BES_numeric[col].cat.codes
#        BES_numeric.loc[ BES_numeric[col]==-1, col ] = np.nan
        

    elif (typ == 6):

        BES_numeric[col].cat.remove_categories( BES_numeric[col].cat.categories.intersection(Weasel_set), inplace=True )
        BES_numeric[col] = BES_numeric[col].astype('float64')

# types
# -1 - anything not a category or one of the below floats
# 0 - personality measures (in steps of .5?),'mapNamesW3','riskScaleW8'
# 1 - linear category, just use  (some made linear by dropping "Weasel_answers")
# 2 - categories need to be modified - order changed
# 3 - set of non-ordered options
# 4 - indirect variables - did someone fill something in in the free text box or not?
# 5 - categories need to modified - things removed
# 6 - categories are integers - should maybe be transformed directly into numbers (mostly "how much money do people need minimum/well off"?)


# load question_categories_correct.csv
# sanity check by type!
# turn into list of variables by type
# 1, 5 handled the same way -> cat.codes
# 6 -> int()
# 4 ignored
# 3 ignored for now (-> vectorized?)
# 2 direct modification




0.0
0.1364256480218281
0.2728512960436562
0.4092769440654843
0.5457025920873124
0.6821282401091405
0.8185538881309686
0.9549795361527967
1.0914051841746248
1.2278308321964528
1.364256480218281
1.500682128240109
generalElectionVote  not in var_type
1.6371077762619373
1.7735334242837653
generalElectionVoteW9  not in var_type
1.9099590723055935
2.0463847203274215
partyId  not in var_type
2.1828103683492497
partyIdSqueeze  not in var_type
2.319236016371078
2.4556616643929057
partyIdEU  not in var_type
2.592087312414734
partyIdW9  not in var_type
2.728512960436562
partyIdSqueezeW9  not in var_type
2.8649386084583903
3.001364256480218
partyIdEUretroW9  not in var_type
3.1377899045020463
3.2742155525238745
3.4106412005457027
3.5470668485675305
3.6834924965893587
3.819918144611187
3.956343792633015
4.092769440654843
4.229195088676671
4.365620736698499
ukCitizen  not in var_type
4.502046384720328
euCitizen  not in var_type
4.638472032742156
commonwealthCitizen  not in var_type
4.774897680763984

In [ ]:
BES_numerics_only = BES_numeric[ BES_numeric.columns[ (var_type["type"]==0) | (var_type["type"]==1) | (var_type["type"]==2) | (var_type["type"]==5) | (var_type["type"]==6)  ] ]

BES_numerics_only.replace(-1,np.nan, inplace=True)
# gender only column that has no nan -> still an int

In [11]:
'id' in var_type["type"].index

True

'partyHelpMethod_5'

In [13]:
var_cat_dict[1]

['responsibleDebtConservativesW1',
 'responsibleDebtLiberalDemsW1',
 'responsibleDebtLabourW1',
 'responsibleDebtUKBanksW1',
 'responsibleDebtGlobalBanksW1',
 'responsibleDebtTradeUnionsW1',
 'responsibleDebtEUW1',
 'responsibleDebtOtherW1',
 'responsibleDebtNoneW1',
 'responsibleEconConservativesW1',
 'responsibleEconLibDemsW1',
 'responsibleEconLabourW1',
 'responsibleEconScottishGovtW1',
 'responsibleEconWelshGovtW1',
 'responsibleEconNoneW1',
 'responsibleNHSConservativesW1',
 'responsibleNHSLibDemsW1',
 'responsibleNHSLabourW1',
 'responsibleNHSScottishGovtW1',
 'responsibleNHSWelshGovtW1',
 'responsibleNHSNoneW1',
 'responsibleEducationConW1',
 'responsibleEducationLDW1',
 'responsibleEducationLabW1',
 'responsibleEducationScotGovtW1',
 'responsibleEducationWelshGovtW1',
 'responsibleEducationNoneW1',
 'responsibleCostLiveConW1',
 'responsibleCostLiveLDW1',
 'responsibleCostLiveLabW1',
 'responsibleCostLiveScotGovtW1',
 'responsibleCostLiveWelshGovtW1',
 'responsibleCostLiveNoneW

In [ ]:
variable_categories = pd.read_csv("question_categories_correct.csv", encoding = "ISO-8859-1")

# flipping list
var_cat_dict = dict()
for typ in [1,2,3,4,5,6]:

    e = variable_categories[variable_categories.type==typ]["column_name"].values

    var_cat_dict[typ] = [item for sublist in [i.split("|") for i in e] for item in sublist]
    
var_cat_dict

In [27]:
var_cat_dict[1]

['responsibleDebtConservativesW1',
 'responsibleDebtLiberalDemsW1',
 'responsibleDebtLabourW1',
 'responsibleDebtUKBanksW1',
 'responsibleDebtGlobalBanksW1',
 'responsibleDebtTradeUnionsW1',
 'responsibleDebtEUW1',
 'responsibleDebtOtherW1',
 'responsibleDebtNoneW1',
 'responsibleEconConservativesW1',
 'responsibleEconLibDemsW1',
 'responsibleEconLabourW1',
 'responsibleEconScottishGovtW1',
 'responsibleEconWelshGovtW1',
 'responsibleEconNoneW1',
 'responsibleNHSConservativesW1',
 'responsibleNHSLibDemsW1',
 'responsibleNHSLabourW1',
 'responsibleNHSScottishGovtW1',
 'responsibleNHSWelshGovtW1',
 'responsibleNHSNoneW1',
 'responsibleEducationConW1',
 'responsibleEducationLDW1',
 'responsibleEducationLabW1',
 'responsibleEducationScotGovtW1',
 'responsibleEducationWelshGovtW1',
 'responsibleEducationNoneW1',
 'responsibleCostLiveConW1',
 'responsibleCostLiveLDW1',
 'responsibleCostLiveLabW1',
 'responsibleCostLiveScotGovtW1',
 'responsibleCostLiveWelshGovtW1',
 'responsibleCostLiveNoneW

In [31]:
import re
# pattern
# \w*? (non-greedy match all alphanumerics)
# (\w*?)_?W[0-9]
re.match('(\w*?)_?W[0-9]' , 'partyHelpMethod_5_W6' ).groups()[0]

new_list = []
for ques in var_cat_dict[1]:
    question = ques
    match_attempt = re.match('(\w*?)_?W[0-9]' , ques )   
    if match_attempt:
        question = match_attempt.groups()[0]
    if question not in new_list:
        new_list.append(question )

        
        
new_list

['responsibleDebtConservatives',
 'responsibleDebtLiberalDems',
 'responsibleDebtLabour',
 'responsibleDebtUKBanks',
 'responsibleDebtGlobalBanks',
 'responsibleDebtTradeUnions',
 'responsibleDebtEU',
 'responsibleDebtOther',
 'responsibleDebtNone',
 'responsibleEconConservatives',
 'responsibleEconLibDems',
 'responsibleEconLabour',
 'responsibleEconScottishGovt',
 'responsibleEconWelshGovt',
 'responsibleEconNone',
 'responsibleNHSConservatives',
 'responsibleNHSLibDems',
 'responsibleNHSLabour',
 'responsibleNHSScottishGovt',
 'responsibleNHSWelshGovt',
 'responsibleNHSNone',
 'responsibleEducationCon',
 'responsibleEducationLD',
 'responsibleEducationLab',
 'responsibleEducationScotGovt',
 'responsibleEducationWelshGovt',
 'responsibleEducationNone',
 'responsibleCostLiveCon',
 'responsibleCostLiveLD',
 'responsibleCostLiveLab',
 'responsibleCostLiveScotGovt',
 'responsibleCostLiveWelshGovt',
 'responsibleCostLiveNone',
 'responsibleImmigConservatives',
 'responsibleImmigLibDems',


In [ ]:
# pattern 2
# what about 
# 'fbInfo_1',
# 'fbInfo_2',
# 'fbInfo_3',

# and
#  'euKnow6',
#  'euKnow2',
#  'euKnow1',
#  'euKnow3',
#  'euKnow4',
#  'euKnow5',


In [52]:
## define 'prune' function to prune wave indicators and return question stubs
## ie. "ptvConW1|ptvLabW1" -> "ptvCon|ptvLab"

x1 = 'incomeWelfareW2'
x2 = 'ptvConW1|ptvLabW1|ptvLDW1|ptvSNPW1|ptvPCW1'


def prune(x):
    
    y= [ ]
    for el in x.split("|"):
        match_attempt = re.match('(\w*?)_?W[0-9]' , el )   
        if match_attempt:
            el = match_attempt.groups()[0]
        y.append(el)
    # should we ditch identical repeats?
    # return "|".join(set(y)) NEEDS TO BE TESTED
    return "|".join(y)
#variable_categories

In [53]:
variable_categories["pruned_column_name"] = variable_categories["column_name"].apply(prune)

In [57]:
# variable_categories.to_csv("temp_name.csv")

In [59]:
# test_df = pd.read_csv("temp_name.csv", encoding = "ISO-8859-1")

In [380]:
## define 'de_Weasel' function to remove Weasel Words from lists of options
## ie. "Yes|No|Don't know" -> "Yes|No"

Weasel_answers = ["Don't know", 'Don?t know', 'Donâ??t know', 'Do\x92t know', 'Dont know', 'Donât know', "Prefer not to say", "Prefer not to answer", "Refused", "Unknown", "Neither", "Other", "I would/will not vote", "Will not vote", "No - not decided", "I would not vote", "It depends", "Other", "Don’t follow politics on Facebook", "Don't follow politics on twitter", "9999.0", "997.0", "222.0", "Yes, other", "Haven't thought about it", "There wasn't a local election in my area", "No, haven't received it", "I don't know what was negotiated", "I never received a response", "There are not local elections in my area", "Can't remember", "Varies too much to say" ]

# non-answer answers
Weasel_set = set(Weasel_answers) # gets rid of duplicates!

# remove weasel phrases
def de_weasel(ques): 

    return "|".join( [x for x in ques.split("|") if x not in Weasel_answers] )

# reorder categories
def re_order(ques):
    if ques in change_cat_dict.keys():
        return "|".join( change_cat_dict[ques] )
    else:
        return ques

def de_num_el(el):
    if el.isdigit():
        el = "%.1f" % int( el )
    return el

def de_number(ques):
    return "|".join( [de_num_el(x) for x in ques.split("|")] )

def floatable(flt):
    try:
        float(flt)
        return True
    except:
        return False

# Weasel_number_answers
def de_weasel_numbers(ques):
    el_list = ques.split("|")
    el_list_len = len(el_list)
    remove_list = []
    for el_pos in range( 0, el_list_len ):

        if el_list[el_pos] in Weasel_number_answers:

            # last element, or not last element but next element is a not a number
            if el_pos==(el_list_len-1) or not floatable(el_list[el_pos+1]):
                remove_list.append(el_list[el_pos])

    return "|".join( [x for x in el_list if x not in remove_list] )

# s.cat.rename_categories([1,2,3])
# EUContactRemainConW8|EUContactRemainLabW8|EUContactRemainLDW8|
# EUContactRemainSNPW8|EUContactRemainPCW8|EUContactRemainUKIPW8|
# EUContactRemainGreenW8|EUContactRemainOthW8|EUContactRemainNoneW8|
# EUContactRemainDKW8|EUContactLeaveConW8|EUContactLeaveLabW8|
# EUContactLeaveLDW8|EUContactLeaveSNPW8|EUContactLeavePCW8|
# EUContactLeaveUKIPW8|EUContactLeaveGreenW8|EUContactLeaveOthW8|
# EUContactLeaveNoneW8|EUContactLeaveDKW8

# pattern match "EUContact*****W8"
# debateOneWatchW8|debateTwoWatchW8

# euRefVoteSqueezeW7 "Will not vote|Yes - Leave|Yes - Remain|No - not decided"
#    -> Stay/remain in the EU|Leave the EU|I would/will not vote|Don't know
#    HMM - RENAME AND REORDER!

# miieuW7
# "Issue stated|Nothing|Don't know" -> "Issue stated|None|Don't know"
# MIIEUW8
# "1.0|Nothing|Don't know" -> "Issue stated|None|Don't know"
# partyIdEUW7|partyIdEUW8
# "Mainly leave|Mainly remain|Fairly evenly split|Don't know" -> "Mainly remain|Fairly evenly divided|Mainly leave|Don't know"
#    HMM - RENAME AND REORDER!

# 1. campaignVisionYesW3|campaignVisionNoW3, govtNatSecuritySuccessW4
# Very unsuccessful|Fairly unsuccessful|Neither successful nor unsuccessful|Fairly successful|Very successful
# Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful

# Fairly <-> Somewhat

# 2. euroTurnoutW1, scotReferendumTurnoutW1|scotReferendumTurnoutW2|welshTurnoutW7|scotTurnoutW7, turnoutUKGeneralW1|turnoutUKGeneralW2|turnoutUKGeneralW3|turnoutUKGeneralW4|turnoutUKGeneralW5|euRefTurnoutW7|euRefTurnoutW8
# Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote
# Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote
# Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote

# "Very unlikely that I vote" <-> "Very unlikely that I will vote" <-> "Very unlikely that I would vote"

rename_cat_dict = {"North East|North West": [ "No", "Yes" ],
                   
                   
                   
                   
                   
                   
                  }

# {"Bad time to buy|Good time to buy|Neither good nor bad time to buy|Don't know": ["Bad time to buy", "Neither good nor bad time to buy", "Good time to buy"],
#                    "Larger|Smaller|About the same|Don't know": ["Larger", "About the same", "Smaller"],

def re_name(ques):
    if ques in rename_cat_dict.keys():
        return "|".join( rename_cat_dict[ques] )
    else:
        return ques







In [402]:
x = 'Not at all British|2|3|4|5|6|Very strongly British|Don\'t know'
x = "Larger|Smaller|About the same|Don't know"
y = "Very unlikely|1|2|3|4|5|6|7|8|9|Very likely"
x = "0 - UK will definitely vote to remain in the EU|1.0|2.0|3.0|4.0|5.0|6.0|7.0|8.0|9.0|10.0|11.0|12.0|13.0|14.0|15.0|16.0|17.0|18.0|19.0|20.0|21.0|22.0|23.0|24.0|25.0|26.0|27.0|28.0|29.0|30.0|31.0|32.0|33.0|34.0|35.0|36.0|37.0|38.0|39.0|40.0|41.0|42.0|43.0|44.0|45.0|46.0|47.0|48.0|49.0|50.0|51.0|52.0|53.0|54.0|55.0|56.0|57.0|58.0|59.0|60.0|61.0|62.0|63.0|64.0|65.0|66.0|67.0|68.0|69.0|70.0|71.0|72.0|73.0|74.0|75.0|76.0|77.0|78.0|79.0|80.0|81.0|82.0|83.0|84.0|85.0|86.0|87.0|88.0|89.0|90.0|91.0|92.0|93.0|94.0|95.0|96.0|97.0|98.0|99.0|100 - UK will definitely vote to leave the EU|Don't know"
x2 = "1.0|2.0|99.0"
x3 = "Should definitely stay in the union|2|3|4|5|6|Should definitely become independent|9999.0"
x4 = "Don't know|15.0|25.0|30.0|50.0|90.0|100.0|120.0|123.0|125.0|150.0|200.0|245.0|250.0|275.0|300.0|350.0|360.0|399.0|400.0|410.0|450.0|500.0|550.0|575.0|600.0|650.0|700.0|715.0|750.0|790.0|799.0|800.0|850.0|880.0|900.0|940.0|950.0|980.0|1000.0|1050.0|1072.0|1100.0|1152.0|1200.0|1250.0|1300.0|1333.0|1350.0|1400.0|1450.0|1500.0|1550.0|1600.0|1650.0|1666.0|1670.0|1700.0|1720.0|1750.0|1800.0|1900.0|1950.0|2000.0|2100.0|2200.0|2250.0|2300.0|2400.0|2500.0|2600.0|2700.0|2750.0|2800.0|2900.0|3000.0|3100.0|3400.0|3500.0|3600.0|3800.0|4000.0|4200.0|4500.0|5000.0|6000.0|7000.0|8000.0|10000.0|10500.0|12000.0|14000.0|15000.0|16000.0|17000.0|18000.0|20000.0|21000.0|22000.0|23000.0|24000.0|25000.0|26000.0|26500.0|28000.0|30000.0|35000.0|36000.0|38000.0|40000.0|43000.0|50000.0"
x5 = "There is a great difference between them|There is some difference|There is not much difference between them|9999.0"
x6 = "0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21.0|22.0|23.0|24.0|25.0|26.0|27.0|28.0|29.0|30.0|31|32|33|34|35|36|37|38|39|40|41|42|43|44|45|46|47|48|49|50|51|52|53|54|55|56|57|58|59|60|61|62|63|64|65|66|67|68|69|70|71|72|73|74|75|76|77|78|79|80|81|82|83|84|85|86|87|88|89|90|91|92|93|94|95|96|99|100|Don't know"
x7 = "The Welsh Assembly|The UK Parliament|Other|222.0|Don't know"
x8 = "Don't know|0.0|4.0|14.0|18.0|23.0|24.0|25.0|40.0|50.0|60.0|74.0|100.0|120.0|130.0|150.0|175.0|200.0|220.0|240.0|250.0"
x9 = "0.0|City of London|Camden|Greenwich|Hackney|Hammersmith and Fulham"
# x = re_order(x)

# re_order (because the matches are exact to original)
# de_weasel_numbers
# de_number
# de_weasel

ques = x2

x = "Mainly leave|Mainly remain|Fairly evenly split|Don't know"
x = "Mainly leave|Mainly remain|Fairly evenly split|Don't know"
re_order(x)
# ques = y
# z2 = de_number(y)
#[x for x in ques.split("|") if x not in Weasel_answers]
#print( "%.1f" % int("5") )

'Mainly remain|Fairly evenly split|Mainly leave'

In [366]:
print(Weasel_number_answers)

['9999.0', '997.0', '222.0', '99.0', '0.0']


In [409]:
variable_categories["de_weaseled_questions"] = variable_categories["question"]
variable_categories["de_weaseled_questions"] = variable_categories["de_weaseled_questions"].apply(re_order)
variable_categories["de_weaseled_questions"] = variable_categories["de_weaseled_questions"].apply(de_weasel_numbers)
variable_categories["de_weaseled_questions"] = variable_categories["de_weaseled_questions"].apply(de_number)
variable_categories["de_weaseled_questions"] = variable_categories["de_weaseled_questions"].apply(de_weasel)
variable_categories["de_weaseled_questions"]

0                                                 No|Yes
1      Strongly disagree|Disagree|Neither agree nor d...
2      Strongly dislike|1.0|2.0|3.0|4.0|5.0|6.0|7.0|8...
3                                                 No|Yes
4      Very unlikely|1.0|2.0|3.0|4.0|5.0|6.0|7.0|8.0|...
5       Not at all certain|Somewhat certain|Very certain
6         Left|1.0|2.0|3.0|4.0|5.0|6.0|7.0|8.0|9.0|Right
7      Very badly|Fairly badly|Neither well nor badly...
8      Unite fully with the European Union|1.0|2.0|3....
9      Government should try to make incomes equal|1....
10       Strongly disagree|Disagree|Agree|Strongly agree
11     Very divided|Fairly divided|Neither united nor...
12     Unification has already gone too far|1.0|2.0|3...
13     Very dissatisfied|A little dissatisfied|Fairly...
14     Getting a lot lower|Getting a little lower|Sta...
15     Not gone nearly far enough|Not gone far enough...
16                  The Welsh Assembly|The UK Parliament
17     Getting a lot worse|Gett

In [410]:
variable_categories["de_weaseled_questions"][130]

'Mainly remain|Fairly evenly split|Mainly leave'

In [411]:
re_order( variable_categories["de_weaseled_questions"][130] )

'Mainly remain|Fairly evenly split|Mainly leave'

In [412]:
x1 = variable_categories["de_weaseled_questions"][0].split("|")
x2 = variable_categories["de_weaseled_questions"][3].split("|")
x3 = variable_categories["de_weaseled_questions"][19].split("|")
x1,x2,x3

(['No', 'Yes'], ['No', 'Yes'], ['Yes', 'No'])

In [413]:
set(x1)==set(x3)

True

In [414]:
## So, I'd like to go through the entire list of *de_weaseled categories*
## and enumerate the unique lists
## On the basis that some will be the same post de_weaseling
## and some will be the same when you ignore ordering

unique_count = 0
unique_list = []
for ind_1 in range(0, len(variable_categories["de_weaseled_questions"].index)):
    
    ques_1 = variable_categories["de_weaseled_questions"][ind_1].encode("ascii", "ignore").decode("utf-8").split("|")
    ques_orig = ques_1
    ques_1 = [x.lower() for x in ques_1]
    
    unique_flag = True
    
    for ind_2 in range(ind_1+1, len(variable_categories["de_weaseled_questions"].index)):
    
        ques_2 = variable_categories["de_weaseled_questions"][ind_2].encode("ascii", "ignore").decode("utf-8").split("|")
        ques_2 = [x.lower() for x in ques_2]
        if ques_1 == ques_2:
            print("Same options: ", ind_1, ind_2, ques_1, ques_2)
            unique_flag = False
        elif set(ques_1) == set(ques_2):
            print("Same options, diff order: ", ind_1, ind_2, ques_1, ques_2)
            unique_flag = False
            
    if unique_flag == True:
        unique_count = unique_count + 1
        unique_list.append(ques_orig)
        
print("unique_count: ", unique_count)

Same options:  0 3 ['no', 'yes'] ['no', 'yes']
Same options, diff order:  0 19 ['no', 'yes'] ['yes', 'no']
Same options:  0 71 ['no', 'yes'] ['no', 'yes']
Same options, diff order:  0 113 ['no', 'yes'] ['yes', 'no']
Same options, diff order:  0 160 ['no', 'yes'] ['yes', 'no']
Same options:  0 297 ['no', 'yes'] ['no', 'yes']
Same options:  1 153 ['strongly disagree', 'disagree', 'neither agree nor disagree', 'agree', 'strongly agree'] ['strongly disagree', 'disagree', 'neither agree nor disagree', 'agree', 'strongly agree']
Same options:  1 208 ['strongly disagree', 'disagree', 'neither agree nor disagree', 'agree', 'strongly agree'] ['strongly disagree', 'disagree', 'neither agree nor disagree', 'agree', 'strongly agree']
Same options, diff order:  3 19 ['no', 'yes'] ['yes', 'no']
Same options:  3 71 ['no', 'yes'] ['no', 'yes']
Same options, diff order:  3 113 ['no', 'yes'] ['yes', 'no']
Same options, diff order:  3 160 ['no', 'yes'] ['yes', 'no']
Same options:  3 297 ['no', 'yes'] ['n

In [239]:
unique_list_non_lower = unique_list

In [241]:
unique_list_lower = unique_list

In [252]:
# case matters in one instance
# A|B|C1|C2|D|E|Refused|Unknown
# a|b|C1|C2|d|e|Refused|Unknown

'a|b|C1|C2|d|e'

In [258]:
len(unique_list)

305

In [256]:
variable_categories["de_weaseled_questions"][1].encode("ascii", "ignore").decode("utf-8")

'Strongly disagree|Disagree|Neither agree nor disagree|Agree|Strongly agree'

In [415]:
unique_list.sort()

unique_df = pd.DataFrame( [list(map("|".join, unique_list)), list(map(len, unique_list))] ).transpose()
unique_df.columns = [ 'answer_set', 'num_answers']
unique_df["answer_set_size"] = unique_df['answer_set'].apply(len)
unique_df.to_csv("unique_answer_set_list.csv")
#list(map("|".join, unique_list))
#list(map(len, unique_list))

#unique_list.to_csv("unique_answer_set_list.csv")

In [390]:
## How do we check that we don't have question_sub collision?

unique_list


[[''],
 ['0 - Scotland will definitely choose to stay in UK',
  '1.0',
  '2.0',
  '3.0',
  '4.0',
  '5.0',
  '6.0',
  '7.0',
  '8.0',
  '9.0',
  '10.0',
  '11.0',
  '12.0',
  '13.0',
  '14.0',
  '15.0',
  '16.0',
  '17.0',
  '18.0',
  '19.0',
  '20.0',
  '21.0',
  '22.0',
  '23.0',
  '24.0',
  '25.0',
  '26.0',
  '27.0',
  '28.0',
  '29.0',
  '30.0',
  '31.0',
  '32.0',
  '33.0',
  '34.0',
  '35.0',
  '36.0',
  '37.0',
  '38.0',
  '39.0',
  '40.0',
  '41.0',
  '42.0',
  '43.0',
  '44.0',
  '45.0',
  '46.0',
  '47.0',
  '48.0',
  '49.0',
  '50.0',
  '51.0',
  '52.0',
  '53.0',
  '54.0',
  '55.0',
  '56.0',
  '57.0',
  '58.0',
  '59.0',
  '60.0',
  '61.0',
  '62.0',
  '63.0',
  '64.0',
  '65.0',
  '66.0',
  '67.0',
  '68.0',
  '69.0',
  '70.0',
  '71.0',
  '72.0',
  '73.0',
  '74.0',
  '75.0',
  '76.0',
  '77.0',
  '78.0',
  '79.0',
  '80.0',
  '81.0',
  '82.0',
  '83.0',
  '84.0',
  '85.0',
  '86.0',
  '87.0',
  '88.0',
  '89.0',
  '90.0',
  '91.0',
  '92.0',
  '93.0',
  '94.0',
  '95.0

In [87]:
list_of_all_col_stubs = []
for ind in variable_categories["pruned_column_name"].index:
    list_of_all_col_stubs.append ( variable_categories["pruned_column_name"][ind].split("|") )
list_of_all_col_stubs

[['responsibleDebtConservatives',
  'responsibleDebtLiberalDems',
  'responsibleDebtLabour',
  'responsibleDebtUKBanks',
  'responsibleDebtGlobalBanks',
  'responsibleDebtTradeUnions',
  'responsibleDebtEU',
  'responsibleDebtOther',
  'responsibleDebtNone',
  'responsibleEconConservatives',
  'responsibleEconLibDems',
  'responsibleEconLabour',
  'responsibleEconScottishGovt',
  'responsibleEconWelshGovt',
  'responsibleEconNone',
  'responsibleNHSConservatives',
  'responsibleNHSLibDems',
  'responsibleNHSLabour',
  'responsibleNHSScottishGovt',
  'responsibleNHSWelshGovt',
  'responsibleNHSNone',
  'responsibleEducationCon',
  'responsibleEducationLD',
  'responsibleEducationLab',
  'responsibleEducationScotGovt',
  'responsibleEducationWelshGovt',
  'responsibleEducationNone',
  'responsibleCostLiveCon',
  'responsibleCostLiveLD',
  'responsibleCostLiveLab',
  'responsibleCostLiveScotGovt',
  'responsibleCostLiveWelshGovt',
  'responsibleCostLiveNone',
  'responsibleImmigConservati

In [83]:
# print ( len( list_of_all_col_stubs ) ) # list length = 3403
# print ( variable_categories["frequency"].sum() ) # matches original doc!
# print (len(set(list_of_all_col_stubs)) ) # but only 1240 unique!

3403

In [416]:
#pos1 = 0
stubs_with_diff_ans_sets = []
for list_pos_1 in range(0,len(list_of_all_col_stubs)):
#var_list in list_of_all_col_stubs: # for each list of questions by sets of shared answer categories
    var_list = list_of_all_col_stubs[list_pos_1]
    for var in set(var_list): # for each *unique* question_stub in those lists
        #pos2 = 0
        for list_pos_2 in range(list_pos_1+1,len(list_of_all_col_stubs)):
        # comp_var_list in list_of_all_col_stubs: # see if it is in other lists
#             if comp_var_list == var_list: # excepting its original list
#                 continue
            if var in list_of_all_col_stubs[list_pos_2]:
                print (list_pos_1, var, list_pos_2)
                if var not in stubs_with_diff_ans_sets:
                    stubs_with_diff_ans_sets.append(var)
                
#             pos2 = pos2 + 1
#     pos1 = pos1+1

0 debateTwoWatch 177
0 partyContactSNP 161
0 selfOccSuperviseLast 3
0 partyContactPC_3 3
0 partyContactSNP_6 3
0 partyContactSNP_3 3
0 partyContactSNP_7 3
0 disability 259
0 partyContactOther_2 3
0 partyContactSNP_4 3
0 partyContactOther_3 3
0 partyContactPC_2 3
0 debateOneWatch 177
0 partyContactOther_4 3
0 partyContactPC_4 3
0 partyContactSNP_2 3
0 partyContactGrn 53
0 partyContactPC_1 3
0 debateThreeWatch 113
0 partyContactOther_7 3
0 partyContactOther_6 3
0 partyContactOther_5 3
0 partyContactPC_7 3
0 partyContactSNP_1 3
0 partyContactPC_6 3
0 partyContactSNP_5 3
0 partyContactNone 3
0 partyContactOther_1 3
0 partyContactPC_5 3
16 devoResponsibleWalesTax 274
19 EUContactRemainNone 24
19 EUContactRemainGreen 24
19 EUContactLeaveCon 24
19 EUContactLeaveNone 24
19 EUContactRemainDK 24
19 EUContactRemainLD 24
19 EUContactLeaveLD 24
19 EUContactRemainSNP 24
19 EUContactLeaveUKIP 24
19 EUContactLeaveLab 24
19 EUContactLeaveSNP 24
19 EUContactRemainPC 24
19 EUContactRemainCon 24
19 EUCont

In [426]:
stubs_with_diff_ans_sets
temp = stubs_with_diff_ans_sets
#difficult = ['partyContact', 'EUContact']
[x for x in temp if x.find("partyContact") and x.find('EUContact') ]
# partyContact
# EUContact
#[n * 2 for n in numbers if n % 2 == 1]

['debateTwoWatch',
 'selfOccSuperviseLast',
 'disability',
 'debateOneWatch',
 'debateThreeWatch',
 'devoResponsibleWalesTax',
 'approveEU',
 'euRefVoteSqueeze',
 'localElectionVote',
 'recallVote15',
 'generalElectionVote',
 'euroElectionVote',
 'partyId',
 'partyIdSqueeze',
 'bestOnMII',
 'discussantturnoutName1',
 'discussantturnoutName2',
 'discussantturnoutName3',
 'bestLeaderCampaign',
 'worstLeaderCampaign',
 'pccVote1',
 'pccVote2',
 'profile_pcon_2010_MyMP_Cal',
 'miilabel',
 'minIncomeGetBy',
 'incomeWelfare',
 'minIncomeWellOff',
 'age',
 'profile_socialgrade_cie',
 'scotRefExpectationTurnout',
 'EUMIICategory']

In [ ]:
# SOME WEIRDNESS NEEDS FIXING

# 161	Don't know	2	10	1	partyContactSNPW1|partyContactSNPW2	4
# 185	1	1	3	1	locusControlW8	4
# 265	No	1	2	1	whichPartiesHelped_99W6	4

# These questions show up as having *only one answer option* 




In [106]:
'partyContactGrn' in list_of_all_col_stubs[0]

True

In [115]:
# BES_Panel["EUContactRemainCon"].cat.categories
# Index(['North East', 'North West'], dtype='object')
# ERROR!




In [100]:
string_0 = 'responsibleDebtConservativesW1|responsibleDebtLiberalDemsW1|responsibleDebtLabourW1|responsibleDebtUKBanksW1|responsibleDebtGlobalBanksW1|responsibleDebtTradeUnionsW1|responsibleDebtEUW1|responsibleDebtOtherW1|responsibleDebtNoneW1|responsibleEconConservativesW1|responsibleEconLibDemsW1|responsibleEconLabourW1|responsibleEconScottishGovtW1|responsibleEconWelshGovtW1|responsibleEconNoneW1|responsibleNHSConservativesW1|responsibleNHSLibDemsW1|responsibleNHSLabourW1|responsibleNHSScottishGovtW1|responsibleNHSWelshGovtW1|responsibleNHSNoneW1|responsibleEducationConW1|responsibleEducationLDW1|responsibleEducationLabW1|responsibleEducationScotGovtW1|responsibleEducationWelshGovtW1|responsibleEducationNoneW1|responsibleCostLiveConW1|responsibleCostLiveLDW1|responsibleCostLiveLabW1|responsibleCostLiveScotGovtW1|responsibleCostLiveWelshGovtW1|responsibleCostLiveNoneW1|responsibleImmigConservativesW1|responsibleImmigLibDemsW1|responsibleImmigLabourW1|responsibleImmigScottishGovtW1|responsibleImmigWelshGovtW1|responsibleImmigNoneW1|responsibleCrimeConservativesW1|responsibleCrimeLibDemsW1|responsibleCrimeLabourW1|responsibleCrimeScottishGovtW1|responsibleCrimeWelshGovtW1|responsibleCrimeNoneW1|conPriorities_econW1|conPriorities_costLiveW1|conPriorities_nhsW1|conPriorities_immigW1|conPriorities_crimeW1|conPriorities_schoolW1|conPriorities_noneW1|labPriorities_econW1|labPriorities_costLiveW1|labPriorities_nhsW1|labPriorities_immigW1|labPriorities_crimeW1|labPriorities_schoolW1|labPriorities_noneW1|ldPriorities_econW1|ldPriorities_costLiveW1|ldPriorities_nhsW1|ldPriorities_immigW1|ldPriorities_crimeW1|ldPriorities_schoolW1|ldPriorities_noneW1|ukipPriorities_econW1|ukipPriorities_costLiveW1|ukipPriorities_nhsW1|ukipPriorities_immigW1|ukipPriorities_crimeW1|ukipPriorities_schoolW1|ukipPriorities_noneW1|partyContact1W1|partyContactConW1|partyContactLabW1|partyContactLDW1|partyContactPCW1|partyContactUKIPW1|partyContactOtherPartyW1|partyContactCon_1W1|partyContactCon_2W1|partyContactCon_3W1|partyContactCon_4W1|partyContactCon_5W1|partyContactCon_6W1|partyContactCon_7W1|partyContactLab_1W1|partyContactLab_2W1|partyContactLab_3W1|partyContactLab_4W1|partyContactLab_5W1|partyContactLab_6W1|partyContactLab_7W1|partyContactLD_1W1|partyContactLD_2W1|partyContactLD_3W1|partyContactLD_4W1|partyContactLD_5W1|partyContactLD_6W1|partyContactLD_7W1|partyContactUKIP_1W1|partyContactUKIP_2W1|partyContactUKIP_3W1|partyContactUKIP_4W1|partyContactUKIP_5W1|partyContactUKIP_6W1|partyContactUKIP_7W1|selfOccSuperviseW1|selfOccSuperviseLastW1|responsibleDebtConservativesW2|responsibleDebtLiberalDemsW2|responsibleDebtLabourW2|responsibleDebtUKBanksW2|responsibleDebtGlobalBanksW2|responsibleDebtTradeUnionsW2|responsibleDebtEUW2|responsibleDebtOtherW2|responsibleDebtNoneW2|responsibleEconConservativesW2|responsibleEconLibDemsW2|responsibleEconLabourW2|responsibleEconScottishGovtW2|responsibleEconWelshGovtW2|responsibleEconNoneW2|responsibleNHSConservativesW2|responsibleNHSLibDemsW2|responsibleNHSLabourW2|responsibleNHSScottishGovtW2|responsibleNHSWelshGovtW2|responsibleNHSNoneW2|responsibleEducationConW2|responsibleEducationLDW2|responsibleEducationLabW2|responsibleEducationScotGovtW2|responsibleEducationWelshGovtW2|responsibleEducationNoneW2|responsibleCostLiveConW2|responsibleCostLiveLDW2|responsibleCostLiveLabW2|responsibleCostLiveScotGovtW2|responsibleCostLiveWelshGovtW2|responsibleCostLiveNoneW2|responsibleImmigConservativesW2|responsibleImmigLibDemsW2|responsibleImmigLabourW2|responsibleImmigScottishGovtW2|responsibleImmigWelshGovtW2|responsibleImmigNoneW2|responsibleCrimeConservativesW2|responsibleCrimeLibDemsW2|responsibleCrimeLabourW2|responsibleCrimeScottishGovtW2|responsibleCrimeWelshGovtW2|responsibleCrimeNoneW2|voteMethodEurope_1W2|voteMethodEurope_2W2|voteMethodEurope_3W2|voteMethodEurope_4W2|discussantsSameAddress_1W2|discussantsSameAddress_2W2|discussantsSameAddress_3W2|discussantsAskedYouToVote_1W2|discussantsAskedYouToVote_2W2|discussantsAskedYouToVote_3W2|discussantsAskedYouToVote_4W2|discussantsAskedYouToVote_5W2|discussantsAskedYouToVote_6W2|discussantsAskedYouToVote_noneW2|discussantsAccompaniedVote_1W2|discussantsAccompaniedVote_2W2|discussantsAccompaniedVote_3W2|discussantsAccompaniedVote_4W2|discussantsAccompaniedVote_5W2|discussantsAccompaniedVote_6W2|discussantsAccompaniedVote_noW2|discussantApprovalVoteName1W2|discussantApprovalVoteName2W2|discussantApprovalVoteName3W2|referendumContact_1W2|referendumContact_2W2|referendumContact_3W2|referendumContact_4W2|referendumContact_5W2|referendumContact_6W2|referendumContact_7W2|referendumContact_8W2|referendumContact_9W2|referendumContact_noneW2|partyContact1W2|partyContactConW2|partyContactLabW2|partyContactLDW2|partyContactPCW2|partyContactUKIPW2|partyContactOtherPartyW2|partyContactCon_1W2|partyContactCon_2W2|partyContactCon_3W2|partyContactCon_4W2|partyContactCon_5W2|partyContactCon_6W2|partyContactCon_7W2|partyContactLab_1W2|partyContactLab_2W2|partyContactLab_3W2|partyContactLab_4W2|partyContactLab_5W2|partyContactLab_6W2|partyContactLab_7W2|partyContactLD_1W2|partyContactLD_2W2|partyContactLD_3W2|partyContactLD_4W2|partyContactLD_5W2|partyContactLD_6W2|partyContactLD_7W2|partyContactPC_1W2|partyContactPC_2W2|partyContactPC_3W2|partyContactPC_4W2|partyContactPC_5W2|partyContactPC_6W2|partyContactPC_7W2|partyContactUKIP_1W2|partyContactUKIP_2W2|partyContactUKIP_3W2|partyContactUKIP_4W2|partyContactUKIP_5W2|partyContactUKIP_6W2|partyContactUKIP_7W2|partyContactOther_1W2|partyContactOther_2W2|partyContactOther_3W2|partyContactOther_4W2|partyContactOther_5W2|partyContactOther_6W2|partyContactOther_7W2|registeredUpToDateW2|reasonNotRegistered_1W2|reasonNotRegistered_2W2|reasonNotRegistered_3W2|reasonNotRegistered_4W2|reasonNotRegistered_5W2|reasonNotRegistered_6W2|reasonNotRegistered_7W2|reasonNotRegistered_8W2|conPriorities_econW2|conPriorities_costLiveW2|conPriorities_nhsW2|conPriorities_immigW2|conPriorities_crimeW2|conPriorities_schoolW2|conPriorities_noneW2|labPriorities_econW2|labPriorities_costLiveW2|labPriorities_nhsW2|labPriorities_immigW2|labPriorities_crimeW2|labPriorities_schoolW2|labPriorities_noneW2|ldPriorities_econW2|ldPriorities_costLiveW2|ldPriorities_nhsW2|ldPriorities_immigW2|ldPriorities_crimeW2|ldPriorities_schoolW2|ldPriorities_noneW2|ukipPriorities_econW2|ukipPriorities_costLiveW2|ukipPriorities_nhsW2|ukipPriorities_immigW2|ukipPriorities_crimeW2|ukipPriorities_schoolW2|ukipPriorities_noneW2|currentUnionMemberW2|everUnionMemberW2|selfOccSuperviseW2|responsibleDebtConservativesW3|responsibleDebtLiberalDemsW3|responsibleDebtLabourW3|responsibleDebtUKBanksW3|responsibleDebtGlobalBanksW3|responsibleDebtTradeUnionsW3|responsibleDebtEUW3|responsibleDebtOtherW3|responsibleDebtNoneW3|responsibleEconConservativesW3|responsibleEconLibDemsW3|responsibleEconLabourW3|responsibleEconScottishGovtW3|responsibleEconWelshGovtW3|responsibleEconNoneW3|responsibleNHSConservativesW3|responsibleNHSLibDemsW3|responsibleNHSLabourW3|responsibleNHSScottishGovtW3|responsibleNHSWelshGovtW3|responsibleNHSNoneW3|responsibleEducationConW3|responsibleEducationLDW3|responsibleEducationLabW3|responsibleEducationScotGovtW3|responsibleEducationWelshGovtW3|responsibleEducationNoneW3|responsibleCostLiveConW3|responsibleCostLiveLDW3|responsibleCostLiveLabW3|responsibleCostLiveScotGovtW3|responsibleCostLiveWelshGovtW3|responsibleCostLiveNoneW3|responsibleImmigConservativesW3|responsibleImmigLibDemsW3|responsibleImmigLabourW3|responsibleImmigScottishGovtW3|responsibleImmigWelshGovtW3|responsibleImmigNoneW3|responsibleCrimeConservativesW3|responsibleCrimeLibDemsW3|responsibleCrimeLabourW3|responsibleCrimeScottishGovtW3|responsibleCrimeWelshGovtW3|responsibleCrimeNoneW3|referendumContact_1W3|referendumContact_2W3|referendumContact_3W3|referendumContact_4W3|referendumContact_5W3|referendumContact_6W3|referendumContact_7W3|referendumContact_8W3|referendumContact_9W3|referendumContact_noneW3|blameScotUnion_cameronW3|blameScotUnion_conW3|blameScotUnion_milibandW3|blameScotUnion_labW3|blameScotUnion_cleggW3|blameScotUnion_libDemW3|blameScotUnion_darlingW3|blameScotUnion_salmondW3|blameScotUnion_snpW3|blameScotUnion_noneW3|partyContact1W3|partyContactConW3|partyContactLabW3|partyContactLDW3|partyContactSNPW3|partyContactPCW3|partyContactUKIPW3|partyContactOtherPartyW3|partyContactCon_1W3|partyContactCon_2W3|partyContactCon_3W3|partyContactCon_4W3|partyContactCon_5W3|partyContactCon_6W3|partyContactCon_7W3|partyContactLab_1W3|partyContactLab_2W3|partyContactLab_3W3|partyContactLab_4W3|partyContactLab_5W3|partyContactLab_6W3|partyContactLab_7W3|partyContactLD_1W3|partyContactLD_2W3|partyContactLD_3W3|partyContactLD_4W3|partyContactLD_5W3|partyContactLD_6W3|partyContactLD_7W3|partyContactSNP_1W3|partyContactSNP_2W3|partyContactSNP_3W3|partyContactSNP_4W3|partyContactSNP_5W3|partyContactSNP_6W3|partyContactSNP_7W3|partyContactUKIP_1W3|partyContactUKIP_2W3|partyContactUKIP_3W3|partyContactUKIP_4W3|partyContactUKIP_5W3|partyContactUKIP_6W3|partyContactUKIP_7W3|registeredUpToDateW3|reasonNotRegistered_1W3|reasonNotRegistered_2W3|reasonNotRegistered_3W3|reasonNotRegistered_4W3|reasonNotRegistered_5W3|reasonNotRegistered_6W3|reasonNotRegistered_7W3|reasonNotRegistered_8W3|conPriorities_econW3|conPriorities_costLiveW3|conPriorities_nhsW3|conPriorities_immigW3|conPriorities_crimeW3|conPriorities_schoolW3|conPriorities_noneW3|labPriorities_econW3|labPriorities_costLiveW3|labPriorities_nhsW3|labPriorities_immigW3|labPriorities_crimeW3|labPriorities_schoolW3|labPriorities_noneW3|ldPriorities_econW3|ldPriorities_costLiveW3|ldPriorities_nhsW3|ldPriorities_immigW3|ldPriorities_crimeW3|ldPriorities_schoolW3|ldPriorities_noneW3|ukipPriorities_econW3|ukipPriorities_costLiveW3|ukipPriorities_nhsW3|ukipPriorities_immigW3|ukipPriorities_crimeW3|ukipPriorities_schoolW3|ukipPriorities_noneW3|currentUnionMemberW3|everUnionMemberW3|selfOccSuperviseW3|selfOccSuperviseLastW3|mapdefined_1W3|mapdefined_2W3|mapdefined_3W3|mapdefined_4W3|mapdefined_5W3|mapdefined_6W3|mapdefined_7W3|mapdefined_8W3|noChanceCoalitionLabW4|noChanceCoalitionConW4|noChanceCoalitionLDW4|noChanceCoalitionSNPW4|noChanceCoalitionPCW4|noChanceCoalitionGrnW4|noChanceCoalitionUKIPW4|noChanceCoalitionNoneW4|responsibleDebtConservativesW4|responsibleDebtLiberalDemsW4|responsibleDebtLabourW4|responsibleDebtUKBanksW4|responsibleDebtGlobalBanksW4|responsibleDebtTradeUnionsW4|responsibleDebtEUW4|responsibleDebtOtherW4|responsibleDebtNoneW4|responsibleEconConservativesW4|responsibleEconLibDemsW4|responsibleEconLabourW4|responsibleEconScottishGovtW4|responsibleEconWelshGovtW4|responsibleEconNoneW4|responsibleNHSConservativesW4|responsibleNHSLibDemsW4|responsibleNHSLabourW4|responsibleNHSScottishGovtW4|responsibleNHSWelshGovtW4|responsibleNHSNoneW4|responsibleEducationConW4|responsibleEducationLDW4|responsibleEducationLabW4|responsibleEducationScotGovtW4|responsibleEducationWelshGovtW4|responsibleEducationNoneW4|responsibleCostLiveConW4|responsibleCostLiveLDW4|responsibleCostLiveLabW4|responsibleCostLiveScotGovtW4|responsibleCostLiveWelshGovtW4|responsibleCostLiveNoneW4|responsibleImmigConservativesW4|responsibleImmigLibDemsW4|responsibleImmigLabourW4|responsibleImmigScottishGovtW4|responsibleImmigWelshGovtW4|responsibleImmigNoneW4|responsibleCrimeConservativesW4|responsibleCrimeLibDemsW4|responsibleCrimeLabourW4|responsibleCrimeScottishGovtW4|responsibleCrimeWelshGovtW4|responsibleCrimeNoneW4|tryReduceImmigConW4|tryReduceImmigLabW4|tryReduceImmigLDW4|tryReduceImmigSNPW4|tryReduceImmigPCW4|tryReduceImmigUKIPW4|tryReduceImmigGrnW4|tryReduceImmigNoneW4|achieveReduceImmigConW4|achieveReduceImmigLabW4|achieveReduceImmigLDW4|achieveReduceImmigSNPW4|achieveReduceImmigPCW4|achieveReduceImmigUKIPW4|achieveReduceImmigGrnW4|achieveReduceImmigNoneW4|tryReduceInequalityConW4|tryReduceInequalityLabW4|tryReduceInequalityLDW4|tryReduceInequalitySNPW4|tryReduceInequalityPCW4|tryReduceInequalityUKIPW4|tryReduceInequalityGrnW4|tryReduceInequalityNoneW4|successReduceInequalityConW4|successReduceInequalityLabW4|successReduceInequalityLDW4|successReduceInequalitySNPW4|successReduceInequalityPCW4|successReduceInequalityUKIPW4|successReduceInequalityGrnW4|successReduceInequalityNoneW4|conAngryW4|conHopeW4|conFearW4|conPrideW4|conNoneW4|labAngryW4|labHopeW4|labFearW4|labPrideW4|labNoneW4|ldAngryW4|ldHopeW4|ldFearW4|ldPrideW4|ldNoneW4|snpAngryW4|snpHopeW4|snpFearW4|snpPrideW4|snpNoneW4|pcAngryW4|pcHopeW4|pcFearW4|pcPrideW4|pcNoneW4|ukipAngryW4|ukipHopeW4|ukipFearW4|ukipPrideW4|ukipNoneW4|grnAngryW4|grnHopeW4|grnFearW4|grnPrideW4|grnNoneW4|partyContact1W4|partyContactConW4|partyContactLabW4|partyContactLDW4|partyContactSNPW4|partyContactPCW4|partyContactUKIPW4|partyContactGrnW4|partyContactOtherPartyW4|partyContactNoneW4|partyContactCon_1W4|partyContactCon_2W4|partyContactCon_3W4|partyContactCon_4W4|partyContactCon_5W4|partyContactCon_6W4|partyContactCon_7W4|partyContactLab_1W4|partyContactLab_2W4|partyContactLab_3W4|partyContactLab_4W4|partyContactLab_5W4|partyContactLab_6W4|partyContactLab_7W4|partyContactLD_1W4|partyContactLD_2W4|partyContactLD_3W4|partyContactLD_4W4|partyContactLD_5W4|partyContactLD_6W4|partyContactLD_7W4|partyContactPC_1W4|partyContactPC_2W4|partyContactPC_3W4|partyContactPC_4W4|partyContactPC_5W4|partyContactPC_6W4|partyContactPC_7W4|partyContactUKIP_1W4|partyContactUKIP_2W4|partyContactUKIP_3W4|partyContactUKIP_4W4|partyContactUKIP_5W4|partyContactUKIP_6W4|partyContactUKIP_7W4|partyContactGreen_1W4|partyContactGreen_2W4|partyContactGreen_3W4|partyContactGreen_4W4|partyContactGreen_5W4|partyContactGreen_6W4|partyContactGreen_7W4|partyContactOther_1W4|partyContactOther_2W4|partyContactOther_3W4|partyContactOther_4W4|partyContactOther_5W4|partyContactOther_6W4|partyContactOther_7W4|registeredUpToDateW4|reasonNotRegistered_1W4|reasonNotRegistered_2W4|reasonNotRegistered_3W4|reasonNotRegistered_4W4|reasonNotRegistered_5W4|reasonNotRegistered_6W4|reasonNotRegistered_7W4|reasonNotRegistered_8W4|referendumSettledW4|conPriorities_econW4|conPriorities_costLiveW4|conPriorities_nhsW4|conPriorities_immigW4|conPriorities_crimeW4|conPriorities_schoolW4|conPriorities_noneW4|labPriorities_econW4|labPriorities_costLiveW4|labPriorities_nhsW4|labPriorities_immigW4|labPriorities_crimeW4|labPriorities_schoolW4|labPriorities_noneW4|ldPriorities_econW4|ldPriorities_costLiveW4|ldPriorities_nhsW4|ldPriorities_immigW4|ldPriorities_crimeW4|ldPriorities_schoolW4|ldPriorities_noneW4|ukipPriorities_econW4|ukipPriorities_costLiveW4|ukipPriorities_nhsW4|ukipPriorities_immigW4|ukipPriorities_crimeW4|ukipPriorities_schoolW4|ukipPriorities_noneW4|discussantsSameAddress_1W4|discussantsSameAddress_2W4|discussantsSameAddress_3W4|discussantApprovalVoteName1W4|discussantApprovalVoteName2W4|discussantApprovalVoteName3W4|conPriorities2_econW4|conPriorities2_costLiveW4|conPriorities2_nhsW4|conPriorities2_immigW4|conPriorities2_crimeW4|conPriorities2_schoolW4|conPriorities2_noneW4|labPriorities2_econW4|labPriorities2_costLiveW4|labPriorities2_nhsW4|labPriorities2_immigW4|labPriorities2_crimeW4|labPriorities2_schoolW4|labPriorities2_noneW4|ldPriorities2_econW4|ldPriorities2_costLiveW4|ldPriorities2_nhsW4|ldPriorities2_immigW4|ldPriorities2_crimeW4|ldPriorities2_schoolW4|ldPriorities2_noneW4|ukipPriorities2_econW4|ukipPriorities2_costLiveW4|ukipPriorities2_nhsW4|ukipPriorities2_immigW4|ukipPriorities2_crimeW4|ukipPriorities2_schoolW4|ukipPriorities2_noneW4|contactMPW4|twitterUseW4|fbUseW4|visitPartySiteW4|partySiteConW4|partySiteLabW4|partySiteLDW4|partySiteSNPW4|partySitePCW4|partySiteUKIPW4|partySiteGreenW4|partySiteOthW4|sharedContentOnline_1W4|sharedContentOnline_2W4|sharedContentOnline_3W4|sharedContentOnline_4W4|sharedContentOnline_5W4|signedUpPartyW4|currentUnionMemberW4|everUnionMemberW4|selfOccSuperviseW4|selfOccSuperviseLastW4|postalTurnoutW5|debateOneWatchW5|debateTwoWatchW5|debateThreeWatchW5|noChanceCoalitionLabW5|noChanceCoalitionConW5|noChanceCoalitionLDW5|noChanceCoalitionSNPW5|noChanceCoalitionPCW5|noChanceCoalitionGrnW5|noChanceCoalitionUKIPW5|noChanceCoalitionNoneW5|partyContact1W5|partyContactConW5|partyContactLabW5|partyContactLDW5|partyContactSNPW5|partyContactPCW5|partyContactUKIPW5|partyContactGrnW5|partyContactOtherPartyW5|partyContactNoneW5|partyContactCon_1W5|partyContactCon_2W5|partyContactCon_3W5|partyContactCon_4W5|partyContactCon_5W5|partyContactCon_6W5|partyContactCon_7W5|partyContactLab_1W5|partyContactLab_2W5|partyContactLab_3W5|partyContactLab_4W5|partyContactLab_5W5|partyContactLab_6W5|partyContactLab_7W5|partyContactLD_1W5|partyContactLD_2W5|partyContactLD_3W5|partyContactLD_4W5|partyContactLD_5W5|partyContactLD_6W5|partyContactLD_7W5|partyContactSNP_1W5|partyContactSNP_2W5|partyContactSNP_3W5|partyContactSNP_4W5|partyContactSNP_5W5|partyContactSNP_6W5|partyContactSNP_7W5|partyContactPC_1W5|partyContactPC_2W5|partyContactPC_3W5|partyContactPC_4W5|partyContactPC_5W5|partyContactPC_6W5|partyContactPC_7W5|partyContactUKIP_1W5|partyContactUKIP_2W5|partyContactUKIP_3W5|partyContactUKIP_4W5|partyContactUKIP_5W5|partyContactUKIP_6W5|partyContactUKIP_7W5|partyContactGreen_1W5|partyContactGreen_2W5|partyContactGreen_3W5|partyContactGreen_4W5|partyContactGreen_5W5|partyContactGreen_6W5|partyContactGreen_7W5|partyContactOther_1W5|partyContactOther_2W5|partyContactOther_3W5|partyContactOther_4W5|partyContactOther_5W5|partyContactOther_6W5|partyContactOther_7W5|participation_1W5|participation_2W5|participation_3W5|participation_4W5|participation_5W5|participation_6W5|twitterUseW5|fbUseW5|visitPartySiteW5|partySiteConW5|partySiteLabW5|partySiteLDW5|partySiteSNPW5|partySitePCW5|partySiteUKIPW5|partySiteGreenW5|partySiteOthW5|sharedContentOnline_1W5|sharedContentOnline_2W5|sharedContentOnline_3W5|sharedContentOnline_4W5|sharedContentOnline_5W5|signedUpPartyW5|regretsIHaveAFewW6|voteMethod_1W6|voteMethod_2W6|voteMethod_5W6|voteMethod_3W6|voteMethod_4W6|participation_1W6|participation_2W6|participation_3W6|participation_4W6|conAngryW6|conHopeW6|conFearW6|conPrideW6|conNoneW6|labAngryW6|labHopeW6|labFearW6|labPrideW6|labNoneW6|ldAngryW6|ldHopeW6|ldFearW6|ldPrideW6|ldNoneW6|snpAngryW6|snpHopeW6|snpFearW6|snpPrideW6|snpNoneW6|pcAngryW6|pcHopeW6|pcFearW6|pcPrideW6|pcNoneW6|ukipAngryW6|ukipHopeW6|ukipFearW6|ukipPrideW6|ukipNoneW6|grnAngryW6|grnHopeW6|grnFearW6|grnPrideW6|grnNoneW6|partyContact1W6|partyContactConW6|partyContactLabW6|partyContactLDW6|partyContactSNPW6|partyContactPCW6|partyContactUKIPW6|partyContactGrnW6|partyContactOtherPartyW6|partyContactNoneW6|partyContactCon_1W6|partyContactCon_2W6|partyContactCon_3W6|partyContactCon_4W6|partyContactCon_5W6|partyContactCon_6W6|partyContactCon_7W6|partyContactLab_1W6|partyContactLab_2W6|partyContactLab_3W6|partyContactLab_4W6|partyContactLab_5W6|partyContactLab_6W6|partyContactLab_7W6|partyContactLD_1W6|partyContactLD_2W6|partyContactLD_3W6|partyContactLD_4W6|partyContactLD_5W6|partyContactLD_6W6|partyContactLD_7W6|partyContactSNP_1W6|partyContactSNP_2W6|partyContactSNP_3W6|partyContactSNP_4W6|partyContactSNP_5W6|partyContactSNP_6W6|partyContactSNP_7W6|partyContactPC_1W6|partyContactPC_2W6|partyContactPC_3W6|partyContactPC_4W6|partyContactPC_5W6|partyContactPC_6W6|partyContactPC_7W6|partyContactUKIP_1W6|partyContactUKIP_2W6|partyContactUKIP_3W6|partyContactUKIP_4W6|partyContactUKIP_5W6|partyContactUKIP_6W6|partyContactUKIP_7W6|partyContactGreen_1W6|partyContactGreen_2W6|partyContactGreen_3W6|partyContactGreen_4W6|partyContactGreen_5W6|partyContactGreen_6W6|partyContactGreen_7W6|partyContactOther_1W6|partyContactOther_2W6|partyContactOther_3W6|partyContactOther_4W6|partyContactOther_5W6|partyContactOther_6W6|partyContactOther_7W6|registeredUpToDateW6|reasonNotRegistered_1W6|reasonNotRegistered_2W6|reasonNotRegistered_3W6|reasonNotRegistered_4W6|reasonNotRegistered_5W6|reasonNotRegistered_6W6|reasonNotRegistered_7W6|reasonNotRegistered_8W6|referendumSettledW6|nonelecParticipation_1W6|nonelecParticipation_2W6|nonelecParticipation_3W6|nonelecParticipation_4W6|nonelecParticipation_5W6|nonelecParticipation_6W6|nonelecParticipation_7W6|nonelecParticipation_8W6|disabilityW6|allSourceIncome_1W6|allSourceIncome_2W6|allSourceIncome_3W6|allSourceIncome_4W6|allSourceIncome_5W6|allSourceIncome_6W6|allSourceIncome_7W6|allSourceIncome_8W6|allSourceIncome_9W6|allSourceIncome_10W6|allSourceIncome_11W6|conPriorities_econW6|conPriorities_costLiveW6|conPriorities_nhsW6|conPriorities_immigW6|conPriorities_crimeW6|conPriorities_schoolW6|conPriorities_noneW6|labPriorities_econW6|labPriorities_costLiveW6|labPriorities_nhsW6|labPriorities_immigW6|labPriorities_crimeW6|labPriorities_schoolW6|labPriorities_noneW6|ldPriorities_econW6|ldPriorities_costLiveW6|ldPriorities_nhsW6|ldPriorities_immigW6|ldPriorities_crimeW6|ldPriorities_schoolW6|ldPriorities_noneW6|ukipPriorities_econW6|ukipPriorities_costLiveW6|ukipPriorities_nhsW6|ukipPriorities_immigW6|ukipPriorities_crimeW6|ukipPriorities_schoolW6|ukipPriorities_noneW6|partyHelpW6|partyHelpMethod_1W6|partyHelpMethod_2W6|partyHelpMethod_3W6|partyHelpMethod_4W6|partyHelpMethod_5W6|twitterUseW6|twitterInfof2fW6|fbUseW6|fbInfof2fW6|selfOccSuperviseW6|selfOccSuperviseLastW6|euSources_1W7|euSources_2W7|euSources_3W7|euSources_4W7|euSources_5W7|euSources_111W7|partyContact1W7|partyContactConW7|partyContactLabW7|partyContactLDW7|partyContactSNPW7|partyContactPCW7|partyContactUKIPW7|partyContactGrnW7|partyContactOtherPartyW7|partyContactStrongerInW7|partyContactVoteLeaveW7|partyContactOthLeaveW7|partyContactOthRemainW7|partyContactNoneW7|partyContactCon_1W7|partyContactCon_2W7|partyContactCon_3W7|partyContactCon_4W7|partyContactCon_5W7|partyContactCon_6W7|partyContactCon_7W7|partyContactLab_1W7|partyContactLab_2W7|partyContactLab_3W7|partyContactLab_4W7|partyContactLab_5W7|partyContactLab_6W7|partyContactLab_7W7|partyContactLD_1W7|partyContactLD_2W7|partyContactLD_3W7|partyContactLD_4W7|partyContactLD_5W7|partyContactLD_6W7|partyContactLD_7W7|partyContactSNP_1W7|partyContactSNP_2W7|partyContactSNP_3W7|partyContactSNP_4W7|partyContactSNP_5W7|partyContactSNP_6W7|partyContactSNP_7W7|partyContactPC_1W7|partyContactPC_2W7|partyContactPC_3W7|partyContactPC_4W7|partyContactPC_5W7|partyContactPC_6W7|partyContactPC_7W7|partyContactUKIP_1W7|partyContactUKIP_2W7|partyContactUKIP_3W7|partyContactUKIP_4W7|partyContactUKIP_5W7|partyContactUKIP_6W7|partyContactUKIP_7W7|partyContactGreen_1W7|partyContactGreen_2W7|partyContactGreen_3W7|partyContactGreen_4W7|partyContactGreen_5W7|partyContactGreen_6W7|partyContactGreen_7W7|partyContactOther_1W7|partyContactOther_2W7|partyContactOther_3W7|partyContactOther_4W7|partyContactOther_5W7|partyContactOther_6W7|partyContactOther_7W7|achieveReduceImmigConW7|achieveReduceImmigLabW7|achieveReduceImmigLDW7|achieveReduceImmigSNPW7|achieveReduceImmigPCW7|achieveReduceImmigUKIPW7|achieveReduceImmigGrnW7|achieveReduceImmigNoneW7|registeredUpToDateW7|reasonNotRegistered_1W7|reasonNotRegistered_2W7|reasonNotRegistered_3W7|reasonNotRegistered_4W7|reasonNotRegistered_5W7|reasonNotRegistered_6W7|reasonNotRegistered_7W7|reasonNotRegistered_8W7|referendumSettledW7|resourceAccess1_1W7|resourceAccess1_2W7|resourceAccess1_3W7|resourceAccess1_4W7|resourceAccess1_5W7|resourceAccess1_6W7|resourceAccess1_111W7|resourceAccess2_1W7|resourceAccess2_2W7|resourceAccess2_3W7|resourceAccess2_4W7|resourceAccess2_111W7|resourceAccess3_1W7|resourceAccess3_2W7|resourceAccess3_3W7|resourceAccess3_4W7|resourceAccess3_5W7|resourceAccess3_6W7|resourceAccess3_7W7|resourceAccess3_8W7|resourceAccess3_111W7|currentUnionMemberW7|everUnionMemberW7|selfOccSuperviseW7|selfOccSuperviseLastW7|postalTurnoutW8|euSources_1W8|euSources_2W8|euSources_3W8|euSources_4W8|euSources_5W8|euSources_111W8|partyContact1W8|partyContactConW8|partyContactLabW8|partyContactLDW8|partyContactSNPW8|partyContactPCW8|partyContactUKIPW8|partyContactGrnW8|partyContactOtherPartyW8|partyContactStrongerInW8|partyContactVoteLeaveW8|partyContactOthLeaveW8|partyContactOthRemainW8|partyContactNoneW8|partyContactCon_1W8|partyContactCon_2W8|partyContactCon_3W8|partyContactCon_4W8|partyContactCon_5W8|partyContactCon_6W8|partyContactCon_7W8|partyContactLab_1W8|partyContactLab_2W8|partyContactLab_3W8|partyContactLab_4W8|partyContactLab_5W8|partyContactLab_6W8|partyContactLab_7W8|partyContactLD_1W8|partyContactLD_2W8|partyContactLD_3W8|partyContactLD_4W8|partyContactLD_5W8|partyContactLD_6W8|partyContactLD_7W8|partyContactSNP_1W8|partyContactSNP_2W8|partyContactSNP_3W8|partyContactSNP_4W8|partyContactSNP_5W8|partyContactSNP_6W8|partyContactSNP_7W8|partyContactPC_1W8|partyContactPC_2W8|partyContactPC_3W8|partyContactPC_4W8|partyContactPC_5W8|partyContactPC_6W8|partyContactPC_7W8|partyContactUKIP_1W8|partyContactUKIP_2W8|partyContactUKIP_3W8|partyContactUKIP_4W8|partyContactUKIP_5W8|partyContactUKIP_6W8|partyContactUKIP_7W8|partyContactGreen_1W8|partyContactGreen_2W8|partyContactGreen_3W8|partyContactGreen_4W8|partyContactGreen_5W8|partyContactGreen_6W8|partyContactGreen_7W8|partyContactOther_1W8|partyContactOther_2W8|partyContactOther_3W8|partyContactOther_4W8|partyContactOther_5W8|partyContactOther_6W8|partyContactOther_7W8|registeredUpToDateW8|reasonNotRegistered_1W8|reasonNotRegistered_2W8|reasonNotRegistered_3W8|reasonNotRegistered_4W8|reasonNotRegistered_5W8|reasonNotRegistered_6W8|reasonNotRegistered_7W8|reasonNotRegistered_8W8|participation_1W8|participation_2W8|participation_3W8|participation_4W8|participation_5W8|participation_6W8|livedAbroadW8|abroadCountry_1W8|abroadCountry_2W8|friendsEUW8|parentsForeignW8|languageSkillsW8|languageSkillsWelshW8|socialDes_1|socialDes_2|socialDes_3|socialDes_4|socialDes_none|privatePrimarySchool|privateSecondarySchool|neverPrivateSchool'

In [102]:
list_0 = prune(string_0)

In [278]:
BES_Panel["EUContactRemainCon"]

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7        North West
8        North East
9               NaN
10              NaN
11              NaN
12              NaN
13              NaN
14              NaN
15              NaN
16              NaN
17              NaN
18              NaN
19              NaN
20              NaN
21              NaN
22              NaN
23              NaN
24              NaN
25       North West
26              NaN
27              NaN
28              NaN
29       North East
            ...    
31379           NaN
31380           NaN
31381           NaN
31382           NaN
31383           NaN
31384           NaN
31385           NaN
31386           NaN
31387           NaN
31388    North West
31389           NaN
31390           NaN
31391           NaN
31392           NaN
31393           NaN
31394    North East
31395           NaN
31396           NaN
31397           NaN


In [437]:

BES_Panel["EUContactRemainLab"].value_counts()

North East    1443
North West     702
Name: EUContactRemainLab, dtype: int64

In [2]:
BES = pd.read_stata("../BES_analysis_data/BES2015_W8_Panel_v1.3.dta")

In [33]:
BES["miiW1"].value_counts()

Issue stated    26686
Don't know       3002
None              666
Name: miiW1, dtype: int64

In [32]:
BES["miiW2"].value_counts()

Issue stated    26772
Don't know       2906
None              558
Name: miiW2, dtype: int64

In [31]:
BES["miiW3"].value_counts()

Issue stated    24448
Don't know       2901
None              477
Name: miiW3, dtype: int64

In [30]:
BES["miiW4"].value_counts()

Issue stated    28236
Don't know       2739
None              432
Name: miiW4, dtype: int64

In [29]:
BES["miiW5"].value_counts()

Issue stated    27913
Don't know       2441
None              437
Name: miiW5, dtype: int64

In [28]:
BES["miiW6"].value_counts()

Issue stated    27080
Don't know       2543
None              499
Name: miiW6, dtype: int64

In [27]:
BES["miiW7"].value_counts()

Issue stated    27750
Don't know       2684
None              461
Name: miiW7, dtype: int64

In [26]:
BES["miiW8"].value_counts()

Issue stated    28522
Don't know       2388
None              499
Name: miiW8, dtype: int64

In [25]:
BES["miieuW7"].value_counts()

Issue stated    22600
Don't know       5821
Nothing          2474
Name: miieuW7, dtype: int64

In [51]:
BES["partyContactGrnW2"].value_counts()

No            9081
Don't know     130
Name: partyContactGrnW2, dtype: int64

In [24]:
str.lower("MIIEUW8")

'miieuw8'

In [44]:
BES["locusControlW8"].value_counts()
# 22983
a = BES["locus1W8"].value_counts()


In [47]:
a.values.sum()

22983